# Notebook Overview

This notebook provides an faster testing interface. 

It includes steps to 
 - Check GPU availability
 - Set up the environment, 
 - Define helper functions,
 - Run tests with a given **id** and **context_type**. 

The results are saved for further analysis.

*Make sure to select the created `venv` as your kernel.*

If there are any errors while running the tests, restart the the notebook, and run again.

## Check GPU Availability
Run this cell to check if there is free space on the gpu. 

If the **free space** is not close to the **total space** then someone else is probably using the machine. 

You can check the active processes bu running `nvidia-smi` in the terminal.

In [1]:
import torch

def get_gpu_memory_torch():
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            free_mem = torch.cuda.mem_get_info(i)[0] / 1024**3
            total_mem = torch.cuda.get_device_properties(i).total_memory / 1024**3
            print(f"GPU {i}: {free_mem:.4f} GiB free / {total_mem:.4f} GiB total")
    else:
        print("No CUDA-compatible GPU detected.")

get_gpu_memory_torch()

GPU 0: 23.2003 GiB free / 23.5746 GiB total


## Setup
The code below imports necessary modules, defines directory paths, and sets the Hugging Face cache path.

**No need to modify this.**

In [2]:
import os
import sys
import json
from pathlib import Path

PROJECT_DIR = Path.cwd()
HAYSTACK_DIR = PROJECT_DIR / "haystack"
RELEVANT_DIR = HAYSTACK_DIR / "relevant"
IRRELAVANT_DIR = HAYSTACK_DIR / "irrelevant"
MISLEADING_IN_RELEVANT_DIR = HAYSTACK_DIR / "misleading_in_relevant"
MISLEADING_IN_IRRELEVANT_DIR = HAYSTACK_DIR / "misleading_in_irrelevant"

sys.path.append(str(PROJECT_DIR))

with open(HAYSTACK_DIR / "needles.json", "r") as f:
    NEEDLES_DATA = json.load(f)

# This sets the Hugging Face cache path. Make sure this directory exists. If not, refer to the README.
os.environ['HF_HOME'] = '.cache/hf_with_quota'

## Helper Functions

In [3]:
from transformers import AutoTokenizer

def load_text(path):
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()

def insert_strings_at_sentence_breaks(text, insert_strings, context_length, tokenizer):
    # Step 1: Tokenize and truncate
    tokens = tokenizer.encode(text, add_special_tokens=False)[:context_length]
    
    # Step 2: Detect sentence breaks by decoding each token
    sentence_break_indices = [
        i for i, tok in enumerate(tokens)
        if tokenizer.decode([tok]).strip().endswith((".", "!", "?"))
    ]

    if len(sentence_break_indices) < len(insert_strings):
        raise ValueError(f"Only {len(sentence_break_indices)} sentence breaks found, "
                         f"but {len(insert_strings)} insertions requested.")

    # Step 3: Compute evenly spaced sentence break positions
    step = len(sentence_break_indices) // (len(insert_strings) + 1)
    insert_positions = [sentence_break_indices[(i + 1) * step] + 1 for i in range(len(insert_strings))]

    # Step 4: Insert insert_strings as tokens
    for pos, insert_str in reversed(list(zip(insert_positions, insert_strings))):
        insert_tokens = tokenizer.encode(insert_str, add_special_tokens=False)
        tokens[pos:pos] = insert_tokens

    return tokenizer.decode(tokens)


def insert_misleading_statements(filepath, insert_strings, context_length, output_path):
    """
    Insert misleading statements into the text at sentence breaks.
    Evenly distribute the misleading statements across the text within the context length.
    """
    tokenizer = AutoTokenizer.from_pretrained("yaofu/llama-2-7b-80k")

    full_text = load_text(filepath)
    modified_text = insert_strings_at_sentence_breaks(full_text, insert_strings, context_length, tokenizer)

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(modified_text)

    print(f"✅ Output saved to {output_path}")


/cs/student/projects1/2021/jiajfang/SNLP_Project/venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from argparse import Namespace

def get_haystack_file(item, context_type, with_misleading, context_length):
    """
    Get the haystack file for the given item and context type.
    Arguments:
        item (dict): The item to process.
        context_type (str): The type of context ("relevant" or "irrelevant").
        with_misleading (bool): Whether to include misleading information.
    Returns:
        str: The path to the haystack file.
    """

    if context_type == "relevant":
        original_dir = RELEVANT_DIR
        original_file = item["context_relevant"]  # Original file without misleading info
        misleading_dir = MISLEADING_IN_RELEVANT_DIR  # Output dir for the misleading file
    elif context_type == "irrelevant":
        original_dir = IRRELAVANT_DIR
        original_file = item["context_irrelevant"]
        misleading_dir = MISLEADING_IN_IRRELEVANT_DIR
    else:
        raise ValueError(f"Unknown context_type: {context_type}")

    if with_misleading:
        haystack_file = misleading_dir / original_file
        insert_misleading_statements(
            filepath=original_dir / original_file,           # Path to the original context file
            insert_strings=item["statements_misleading"],    # List of misleading statements
            context_length=context_length,                   # Max context length for insertion
            output_path=haystack_file                        # Output path for the processed file
        )
    else:
        haystack_file = original_dir / original_file

    return haystack_file

def get_args(id, context_type, with_misleading, data=NEEDLES_DATA):
    """
    Get the arguments for the given id and context type.
    Arguments:
        id (int): The id of the item.
        context_type (str): The type of context ("relevant" or "irrelevant").
        with_misleading (bool): Whether to include misleading information.
        data (list): The list of data items.
    Returns:
        Namespace: The arguments for the given id and context type.
    """

    context_length_max = 5000
    
    # find the item with the given id
    item = next(item for item in data if item["id"] == id)

    # get the haystack file for the given item and context type
    haystack_file = get_haystack_file(item, context_type, with_misleading, context_length_max)

    args = Namespace(
        model_name = "yaofu/llama-2-7b-80k",
        model_name_suffix = ( f"id_{item['id']}_{context_type}" if not with_misleading 
                             else f"id_{item['id']}_{context_type}_misleading"), # suffix used to name the results files,
        model_provider = "LLaMA",

        context_lengths_min = 0, # min context length
        context_lengths_max = context_length_max, # max context length
        context_lengths_num_intervals = 5, # number of intervals for context lengths

        document_depth_percent_intervals = 5, # number of intervals for document depth

        needle = item["needle_refined"],
        real_needle = item["real_needle_refined"],
        retrieval_question = item["question_refined"],
        haystack_file = haystack_file,
    )

    return args

In [5]:
import gc
import torch
from retrieval_head_detection import LLMNeedleHaystackTester as RetrievalHeads

def cleanup(tester: RetrievalHeads):
    del tester.model_to_test
    del tester.testing_results
    del tester.head_counter
    del tester
    
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

def run_test(args):
    try:
        tester = RetrievalHeads(**vars(args))
        tester.start_test()
    finally:
        cleanup(tester)

In [6]:
import sys
print("Python executable path:")
print(sys.executable)

print("\nPython environment site-packages:")
print(sys.path)


Python executable path:
/cs/student/projects1/2021/jiajfang/SNLP_Project/venv/bin/python

Python environment site-packages:
['/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/cs/student/projects1/2021/jiajfang/SNLP_Project/venv/lib64/python3.9/site-packages', '/cs/student/projects1/2021/jiajfang/SNLP_Project/venv/lib/python3.9/site-packages', '/cs/student/projects1/2021/jiajfang/SNLP_Project', '/tmp/tmpajuruwil', './faiss_attn/']


## Runing the Tests

### Set up the test object:
Specify the **id** and the **context_type** in the `get_args` function.

If `with_misleading` is True, then misleading statements are added to the specified `context_type` file.

### Start the Test

Running the test will save:

- **Contexts** → Given to the model per context length/depth  
  → `contexts/{model_name}_{context_type}_id_{id}`

- **Results** → Model outputs per context length/depth  
  → `results/graph/{model_name}_{context_type}_id_{id}`

**Example**:  
`results/graph/llama-2-7b-80k_relevant_id_44/`

Each test should take <=5 minutes. It sometimes uses the CPU instead of the GPU, if it is taking longer to finish, then restart the notebook and run again. 

In [13]:
ids = []
for file_name in os.listdir('./haystack/irrelevant'):
    if file_name.endswith('.txt'):
        file_id = os.path.splitext(file_name)[0]  # Extract the file name without extension
        ids.append(file_id)
ids.sort()
id_vals = [int(i) for i in ids[14:28]]
print(id_vals)

[168, 17, 170, 172, 173, 175, 18, 180, 182, 183, 185, 189, 192, 193]


In [14]:
for id in id_vals:
    for context_type in ["relevant", "irrelevant"]:
        for with_misleading in [False, True]:
            args = get_args(id, context_type, with_misleading)
            run_test(args)

loading from yaofu/llama-2-7b-80k
layer number: 32, head number 32


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.12it/s]




Starting Needle In A Haystack Testing...
- Model: yaofu/llama-2-7b-80k
- Context Lengths: 5, Min: 0, Max: 5000
- Document Depths: 5, Min: 0%, Max: 100%
- Needle: The most recent former Prime Minister of the United Kingdom is Rishi Sunak.



insertion at 0
The most recent former Prime Minister of the United Kingdom is Rishi Sunak.
[['6-9'], ['11-15'], ['7-4'], ['16-19'], ['8-26'], ['19-15'], ['17-22'], ['8-22'], ['12-26'], ['21-30'], ['24-29'], ['31-16'], ['11-2'], ['15-14'], ['18-30'], ['19-14'], ['26-3'], ['28-14'], ['0-9'], ['6-30']]
-- Test Summary -- 
Duration: 0.7 seconds
Context: 0 tokens
Depth: 0%
Score: 100.00001192092896
Response: The most recent former Prime Minister of the United Kingdom is Rishi Sunak.

Writing at results/graph/llama-2-7b-80k_id_168_relevant/llama-2-7b-80k_id_168_relevant_len_0_depth_0_results.json
insertion at 0
The most recent former Prime Minister of the United Kingdom is Rishi Sunak.
[['6-9'], ['11-15'], ['7-4'], ['16-19'], ['8-26'], ['19-15'], ['17-2

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.10it/s]




Starting Needle In A Haystack Testing...
- Model: yaofu/llama-2-7b-80k
- Context Lengths: 5, Min: 0, Max: 5000
- Document Depths: 5, Min: 0%, Max: 100%
- Needle: The most recent former Prime Minister of the United Kingdom is Rishi Sunak.



insertion at 0
The most recent former Prime Minister of the United Kingdom is Rishi Sunak.
[['6-9'], ['11-15'], ['7-4'], ['16-19'], ['8-26'], ['19-15'], ['17-22'], ['8-22'], ['12-26'], ['21-30'], ['24-29'], ['31-16'], ['11-2'], ['15-14'], ['18-30'], ['19-14'], ['26-3'], ['28-14'], ['0-9'], ['6-30']]
-- Test Summary -- 
Duration: 0.7 seconds
Context: 0 tokens
Depth: 0%
Score: 100.00001192092896
Response: The most recent former Prime Minister of the United Kingdom is Rishi Sunak.

Writing at results/graph/llama-2-7b-80k_id_168_relevant_misleading/llama-2-7b-80k_id_168_relevant_misleading_len_0_depth_0_results.json
insertion at 0
The most recent former Prime Minister of the United Kingdom is Rishi Sunak.
[['6-9'], ['11-15'], ['7-4'], ['16-19'], ['8-2

...



Starting Needle In A Haystack Testing...
- Model: yaofu/llama-2-7b-80k
- Context Lengths: 5, Min: 0, Max: 5000
- Document Depths: 5, Min: 0%, Max: 100%
- Needle: There are 20 member states in the Eurozone.



insertion at 0
There are 20 member states in the Eurozone.
[['11-15'], ['16-19'], ['6-9'], ['6-30'], ['7-4'], ['8-26'], ['12-26'], ['19-15'], ['21-30'], ['16-24'], ['17-22'], ['21-4'], ['24-29'], ['24-30'], ['7-13'], ['8-31'], ['12-16'], ['13-11'], ['15-14'], ['16-1']]
-- Test Summary -- 
Duration: 0.6 seconds
Context: 0 tokens
Depth: 0%
Score: 100.00001192092896
Response: There are 20 member states in the Eurozone.

Writing at results/graph/llama-2-7b-80k_id_193_relevant_misleading/llama-2-7b-80k_id_193_relevant_misleading_len_0_depth_0_results.json
insertion at 0
There are 20 member states in the Eurozone.
[['11-15'], ['16-19'], ['6-9'], ['6-30'], ['7-4'], ['8-26'], ['12-26'], ['19-15'], ['21-30'], ['16-24'], ['17-22'], ['21-4'], ['24-29'], ['24-30'], ['7-13'], ['8-31'], ['12-

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.18it/s]




Starting Needle In A Haystack Testing...
- Model: yaofu/llama-2-7b-80k
- Context Lengths: 5, Min: 0, Max: 5000
- Document Depths: 5, Min: 0%, Max: 100%
- Needle: There are 20 member states in the Eurozone.



insertion at 0
There are 20 member states in the Eurozone.
[['11-15'], ['16-19'], ['6-9'], ['6-30'], ['7-4'], ['8-26'], ['12-26'], ['19-15'], ['21-30'], ['16-24'], ['17-22'], ['21-4'], ['24-29'], ['24-30'], ['7-13'], ['8-31'], ['12-16'], ['13-11'], ['15-14'], ['16-1']]
-- Test Summary -- 
Duration: 0.6 seconds
Context: 0 tokens
Depth: 0%
Score: 100.00001192092896
Response: There are 20 member states in the Eurozone.

Writing at results/graph/llama-2-7b-80k_id_193_irrelevant/llama-2-7b-80k_id_193_irrelevant_len_0_depth_0_results.json
insertion at 0
There are 20 member states in the Eurozone.
[['11-15'], ['16-19'], ['6-9'], ['6-30'], ['7-4'], ['8-26'], ['12-26'], ['19-15'], ['21-30'], ['16-24'], ['17-22'], ['21-4'], ['24-29'], ['24-30'], ['7-13'], ['8-31'], ['12-16'], ['13-11'], [

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.18it/s]




Starting Needle In A Haystack Testing...
- Model: yaofu/llama-2-7b-80k
- Context Lengths: 5, Min: 0, Max: 5000
- Document Depths: 5, Min: 0%, Max: 100%
- Needle: There are 20 member states in the Eurozone.



insertion at 0
There are 20 member states in the Eurozone.
[['11-15'], ['16-19'], ['6-9'], ['6-30'], ['7-4'], ['8-26'], ['12-26'], ['19-15'], ['21-30'], ['16-24'], ['17-22'], ['21-4'], ['24-29'], ['24-30'], ['7-13'], ['8-31'], ['12-16'], ['13-11'], ['15-14'], ['16-1']]
-- Test Summary -- 
Duration: 0.6 seconds
Context: 0 tokens
Depth: 0%
Score: 100.00001192092896
Response: There are 20 member states in the Eurozone.

Writing at results/graph/llama-2-7b-80k_id_193_irrelevant_misleading/llama-2-7b-80k_id_193_irrelevant_misleading_len_0_depth_0_results.json
insertion at 0
There are 20 member states in the Eurozone.
[['11-15'], ['16-19'], ['6-9'], ['6-30'], ['7-4'], ['8-26'], ['12-26'], ['19-15'], ['21-30'], ['16-24'], ['17-22'], ['21-4'], ['24-29'], ['24-30'], ['7-13'], ['8-31'], [

In [9]:
# Example 1: Relevant context without misleading statements
# args = get_args(id=id_val, context_type="relevant", with_misleading=False)
# run_test(args)

In [10]:
# Example 2: Irrelevant context without misleading statements
# args = get_args(id=id_val, context_type="irrelevant", with_misleading=False)
# run_test(args)

In [11]:
# Exmample 3: Relevant context with misleading statements
# args = get_args(id=id_val, context_type="relevant", with_misleading=True)
# run_test(args)

In [12]:
# Example 4: Irrelevant context with misleading statements
# args = get_args(id=id_val, context_type="irrelevant", with_misleading=True)
# run_test(args)